# Sea surface template_title validation using gridded observations from source_title

In [ ]:
variable = "template_variable" 

In [ ]:
chunk_start

In [ ]:
stamp = nc.session_info["stamp"]
out = ".trackers/" + stamp + ".txt"
if not os.path.exists(".trackers"):
    os.makedirs(".trackers")
# save out as empty file
with open(out, 'w') as f:
    f.write("")

In [ ]:
source = "source_name"
sub_regions = "sub_regions_value"

In [ ]:

ff = glob.glob(f"data_dir_value/oceanval_matchups/gridded/{variable}/{source}_{variable}_*surface.nc")
ff = [x for x in ff if f"{source}_" in x]
ff_def = ff[0].replace(".nc", "_definitions.pkl")
with open(ff_def, "rb") as f:
    definitions = pickle.load(f)
model_variable = definitions[variable].model_variable    
vv_name = definitions[variable].short_name
vv_shortname = definitions[variable].short_name
vv_longname = definitions[variable].long_name   
md(f"**Matchup procedure**: The model and observations were matched up as follows. First, the model dataset was cropped by a small amount to make sure cells close to the boundary were removed. The model was then regridded to the observational grid using bilinear remmaping. Only grid cells with model and observational data were maintained. The following model output was used to compare with the observational values: **{model_variable}**.")
    

In [ ]:
try:
    ff_summary = glob.glob(os.path.dirname(ff[0]) + "/*summary.pkl")[0]
    with open(ff_summary, 'rb') as f:
        vv_summary = pickle.load(f)
    clim_years = summary["clim"]
    clim_years = " (" + str(clim_years[0]) + "-" + str(clim_years[1]) + ") "
except:
    clim_years = ""
    pass

if len(ff) != 1:
    raise ValueError("Something is wrong with the file")
layer = os.path.basename(ff[0]).split("_")[-1].replace(".nc", "")
if layer == "surface":
    layer_long = "sea surface"
ff_vertical = ff[0].replace("_surface.nc", "_vertical.nc")
ds_model = nc.open_data(ff)
ds_model.subset(lon = lon_lim, lat = lat_lim)
# change the long_name
ds_model.set_precision("F32")
ds_model.subset(variable = "model")
ds_model.tmean("month")
ds_model.run()
new_name = definitions[variable].long_name 
new_name = new_name[0].upper() + new_name[1:]   
ds_model.set_longnames({ds_model.variables[0]: new_name})
ds_year = min(ds_model.years)
ds_model.set_year(ds_year)
ds_times = ds_model.times
df_times = pd.DataFrame({"year":[x.year for x in ds_times]}).groupby("year").size().reset_index()
df_times.columns = ["year", "count"]
years = list(df_times.query("count > 1").year)
ds_model.as_missing(0)
# if variable is doc, add 40
ds_model.run()
ds_annual = ds_model.copy()
ds_annual.tmean()
# ds_annual.set_longnames({ds_annual.variables[0]: Variable})

In [ ]:
md("## Baseline climatologies of sea surface template_title")

In [ ]:
# model climatology years can be derived from the vv_summary
try:
    clim_years = vv_summary["clim_years"]
    if clim_years[0] == clim_years[-1]:
        clim_text = f" The model climatology is calculated using the year **{clim_years[0]}**."
        clim_range = f" ({clim_years[0]}) "
    else:
        clim_text = f" The model climatology is calculated using the years **{clim_years[0]}-{clim_years[-1]}**."
        clim_range = f" ({clim_years[0]}-{clim_years[-1]}) "
except:
    clim_text = ""
    clim_range = ""
    pass
md_markdown(f"Climatologies of model and observational {layer_long} {vv_name} are shown in the figures below.{clim_text}")

In [ ]:
ff = glob.glob(f"data_dir_value/oceanval_matchups/gridded/{variable}/**_{variable}_*surface.nc")
ff = [x for x in ff if f"{source}_" in x]
ds_obs = nc.open_data(ff)
ds_obs.subset(variable = "observation")
ds_obs.subset(lon = lon_lim, lat = lat_lim)

ds_obs.run()
# changte the long_name
new_name = definitions[variable].long_name
new_name = new_name[0].upper() + new_name[1:]   
ds_obs.set_longnames({ds_obs.variables[0]: new_name})
ds_obs.set_precision("F32")
ds_obs.tmean("month")
ds_obs.run()

In [ ]:
chunk_clim

In [ ]:
chunk_bias

In [ ]:
n_times = len(ds_obs.times)
if n_times > 0:
    md(f"## Can the model reproduce seasonality of {layer_long} {vv_longname}?")

    md(f"The ability of the model to reproduce seasonality of {layer_long} {vv_name} was assessed by comparing the modelled and observed seasonal cycle of {vv_name}. First, we derive a monthly climatology for the model data. Then, we calculate the Pearson correlation coefficient between the modelled and observed {vv_name} at each grid cell.")

    md("Note: we are only assessing the ability of the model to reproduce the ability of the model to reproduce seasonal changes, not long-term trends.")


In [ ]:
chunk_seasonal

In [ ]:
if concise and sub_regions == "None": 
    regional = False
n_times = len(ds_obs.times)
if n_times < 12:
   regional = False
if regional:
    md(f"## Regional assessment of model performance for {layer_long} {vv_longname}")

In [ ]:
if regional:
    md("We assessed the regional performance of the model by comparing the model with observations in a number of regions. The regions considered are mapped below.")

In [ ]:
if regional:
    lon_name = [x for x in ds_regions.to_xarray().coords if "lon" in x][0]
    lat_name = [x for x in ds_regions.to_xarray().coords if "lat" in x][0]
    df_mapped = (
        ds_regions
        .to_dataframe()
        .reset_index()
        # rename the columns
        .rename(columns = {lon_name: "lon", lat_name: "lat"})
        .melt(id_vars = ["lon", "lat"])
        .dropna()
        .merge(regions_contents.loc[:,["variable", "long_name"]])
        .drop(columns = [ "value"])
    )
    bad = ["Rosa", "Locate Shelf"]
    df_mapped = df_mapped.query("long_name not in @bad")
    xlim = np.array([df_mapped.lon.min(), df_mapped.lon.max()])
    ylim = np.array([df_mapped.lat.min(), df_mapped.lat.max()])

    def fix_name(x):
        x = x.replace("North East", "NE")
        x = x.replace("North ", "N ")
        if x == "Channel":
            x = "English Channel"
        return x

    fix_name = np.vectorize(fix_name)


    df_mapped.long_name = fix_name(df_mapped.long_name)

else:
    df_mapped = 1




In [ ]:
if regional:
    df_all = []
    df_summary = []
    for vv in ds_regions.variables:
        ds_rr = ds_regions.copy()
        ds_rr.subset(variable = vv)
        ds_rr.run()
        ds_vv = ds_ts.copy()
        time_name = [x for x in list(ds_vv.to_xarray().coords) if "time" in x][0]
        ds_vv * ds_rr
        ds_region = ds_vv.copy()
        ds_cor = ds_vv.copy()
        ds_cor.tmean()
        ds_cor.cor_space("model", "observation")
        region = list(regions_contents.query("variable == @vv").long_name)[0]
        df1 = (
            ds_cor
            .to_dataframe()
            .reset_index()
            .loc[:,["cor"]]
            .rename(columns = {"cor": "Spatial correlation"})
            .assign(region = region)
        )
        # now do the temporal correlation

        ds_cor = ds_vv.copy()
        ds_cor.cor_time("model", "observation")
        ds_cor.spatial_mean()
        df2 = (
            ds_cor
            .to_dataframe()
            .reset_index()
            .loc[:,["cor"]]
            .rename(columns = {"cor": "Temporal correlation"})
            .assign(region = region)
        )
        df = df1.merge(df2)

        ds_vv.spatial_mean()
        df_bias = (
            ds_vv
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .assign(month = lambda x: x.time.dt.month)
            .assign(bias = lambda x: x.model - x.observation)
            .assign(region = region)
            .groupby("region")
            .mean()
            .reset_index()
            .loc[:,["region", "bias"]]
        )

        # now add the RMSD, calculated in the same way as the bias
        df_rmsd = (
            ds_region
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .assign(month = lambda x: x.time.dt.month)
            .assign(rmsd = lambda x: (x.model - x.observation)**2)
            .assign(region = region)
            .groupby("region")
            .mean()
            .reset_index()
            .loc[:,["region", "rmsd"]]
            .assign(rmsd = lambda x: np.sqrt(x.rmsd))
        )

        df = df1.merge(df2).merge(df_bias).merge(df_rmsd)
        df_summary.append(df)

        


        df_vv = (
            ds_vv
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .melt("time")
            .assign(month = lambda x: x.time.dt.month)
            .assign(region = vv)
        )
        df_all.append(df_vv)
        ds_region.tmean()
        df_region = (
            ds_region
            .to_dataframe()
            .dropna()
            .reset_index()
            .loc[:,["model", "observation"]]
            .drop_duplicates()
        )
    
        del ds_rr, ds_vv, ds_region
    df_all = pd.concat(df_all).dropna()
        
    df_all = (
        df_all
        .merge(df_mapped.loc[:,["long_name", "variable"]].drop_duplicates().rename(columns = {"variable": "region"}))
    )
    df_summary= pd.concat(df_summary)

In [ ]:
# restrict df_mapped to regions in df_all
if regional:
    regions = set(df_all.query("value > 0").region)
    df_mapped = df_mapped.query("variable in @regions")
else:
    # some quick hacks to prevent an error in the next cell
    df_mapped = "foobar"
    xlim = "foobar"
    ylim = "foobar"
    global_grid = "foobar"

In [ ]:
%%capture --no-display
%%R -i regional -i df_mapped -i xlim -i ylim -i global_grid -w 800 -h 600

if (regional){

    library(tidyverse)

    unique_long_names <- unique(df_mapped$long_name)
    if("Full Domain" %in% unique_long_names){
        unique_long_names <- unique_long_names[unique_long_names != "Full Domain"]
        # add Full Domain, but put it first
        unique_long_names <- c("Full Domain", unique_long_names)
        df_mapped <- df_mapped %>%
            mutate(long_name = factor(long_name, levels = unique_long_names)) 
    }

    world_map <- map_data("world")

    gg <-  ggplot(df_mapped)+
        geom_tile(aes(x = lon, y = lat))+
        coord_cartesian(xlim = xlim, ylim = ylim, expand = FALSE)+
        theme_bw(base_size = 12)+
        facet_wrap(~long_name, ncol = 5)+
        theme(axis.title.x = element_blank(),
              axis.title.y = element_blank())


gg <- gg +
    # remove the x and y axis totally
    theme(axis.text.x = element_blank(), axis.text.y = element_blank(),
          axis.ticks.x = element_blank(), axis.ticks.y = element_blank(),
          axis.title.x = element_blank(), axis.title.y = element_blank()) +
    labs(x = "", y = "") 

gg <- gg + 
        geom_polygon(data = world_map, aes(x = long, y = lat, group = group), fill = "grey", color = "grey")



    gg

}

In [ ]:
if regional:
    md(f"**Figure {i_figure}**: Regions used for validation of {layer_long} {vv_longname}.")
i_figure += 1

In [ ]:
if regional:
    md(f"Time series were constructed comparing the monthly mean of the spatial average {layer_long} {vv_name} in each region. The spatial average was calculated using the mean of all grid cells within each region, accounting for grid cell area.")

In [ ]:
if regional:
    out_ts = f"../../oceanval_results/regionals/{source}_{variable}_regionals.csv"
    # check if directory exists for out_ts
    if not os.path.exists("../../oceanval_results/regionals"):
        os.makedirs("../../oceanval_results/regionals")
    df_all.to_csv(out_ts, index = False)
    # store the defintions
    ff_def = out_ts.replace(".csv", "_definitions.pkl")
    with open(ff_def, "wb") as f:
        pickle.dump(definitions, f)
    #out_ts = f"../../oceanval_results/regionals/{source}_{variable}_regionals.csv"
    # save the region name to an empty txt file
    out_file = f"../../oceanval_results/regionals/{sub_regions}.txt"
    with open(out_file, 'w') as f:
        f.write("")
if not regional:
    df_all = False

if regional:
    try:
        units = model_unit 
    except:
        units = ds_ts.contents.unit[0]
        units = str(units)

else:
    try:
        units = model_unit
    except:
        units = False

In [ ]:
%%capture --no-display
%%R -i df_all -i regional -i vv_name -i units  -w 800 -h 600
variable = vv_name

# do a seasonal plot
if(regional){

    # convert long_name to factor
    # unique_long_names 
    unique_long_names <- unique(df_all$long_name)
    if("Full Domain" %in% unique_long_names){
        unique_long_names <- unique_long_names[unique_long_names != "Full Domain"]
        # add Full Domain, but put it first
        unique_long_names <- c("Full Domain", unique_long_names)
    df_all <- df_all %>%
        mutate(long_name = factor(long_name, levels = unique_long_names)) 
    }

    x_lab = str_glue("Spatial average {variable} ({units})")
    x_lab <- str_replace(x_lab, "/m\\^3", "m<sup>-3</sup>")
    x_lab <- str_replace(x_lab, "/m\\^2", "m<sup>-2</sup>")
    # handl /yr
    x_lab <- str_replace(x_lab, "/yr", " yr<sup>-1</sup>")
    # handle /m2
    x_lab <- str_replace(x_lab, "/m2", "m<sup>-2</sup>")
    # handle /m3
    x_lab <- str_replace(x_lab, "/m3", "m<sup>-3</sup>")
    # fix /kg to kg^-1
    x_lab <- str_replace(x_lab, "/kg", "kg<sup>-1</sup>")
    # CO2
    x_lab <- str_replace(x_lab, "CO2", "CO<sub>2</sub>")
    # O_2
    x_lab <- str_replace(x_lab, "O2", "O<sub>2</sub>")
    x_lab <- str_replace(x_lab, "O_2", "O<sub>2</sub>")
    

    library(tidyverse, warn.conflicts = FALSE)
    library(ggplot2, warn.conflicts = FALSE)
    library(ggthemes, warn.conflicts = FALSE)
    # make variable title
    df_all = df_all %>%
        mutate(variable = str_to_title(variable))

    gg <- ggplot(df_all)+
        geom_line(aes(x = month, y = value, colour = variable))+
        facet_wrap(~long_name, ncol = 5)+
        labs(y = x_lab)+
        labs(x = "Month")+
        theme(legend_position = "top")+
        scale_color_manual(values = c("red", "blue"))+
        scale_x_continuous(breaks = c(1, 4, 7, 10), labels = c("Jan", "Apr", "Jul", "Oct"))+ 
        theme_bw(base_size = 12)+
        labs(colour = "")+
        theme(legend.position = "top")+
        theme(axis.title.y = ggtext::element_markdown())+
        scale_color_fivethirtyeight()

        gg


}



In [ ]:
if regional:
    md(f"**Figure {i_figure}**: Seasonal cycle of {layer_long} {vv_longname} for model and observations for each region. The spatial average is taken over the region.") 
    i_figure += 1

In [ ]:
if regional:
    md(f"The table below shows the average bias of sea surface {vv_name} in each region. The bias is calculated as the modelled value minus the observed value. A positive bias indicates that the model overestimates the observed value, while a negative bias indicates that the model underestimates the observed value.")

if regional:
    # output and hide index
    # first average things and tidy
    df_summary = (
        df_summary
        # .loc[:,["region", "bias"]]
        .groupby("region")
        .mean()
        .reset_index()
        # capitalize the columns
        .rename(columns = {"region": "Region", "bias": "Bias", "rmsd": "RMSD"})
    )
    # remove na values
    df_summary = df_summary.dropna().reset_index(drop = True)
    if not global_grid:
        # drop the spatial correlation
        df_summary = df_summary.drop(columns = ["Spatial correlation"])
    df_display(df_summary)


In [ ]:
if regional:
    regional_summary = [f"**Table {i_table}**: Summary of performance of the model sea surface {vv_longname} in each region."]
    regional_summary.append(f"The bias ({model_unit}) column represents the spatial average of the annual mean modelled value minus the observed value.")
    regional_summary.append("The temporal correlation column represents the spatial mean of the temporal correlation between the model and observations per grid cell.") 
    if global_grid:
        regional_summary.append("The spatial correlation column represents the spatial correlation between the model and observations.")
    regional_summary = " ".join(regional_summary).replace("  ", " ")
    md(regional_summary)
    i_table = i_table + 1

In [ ]:
chunk_results

In [ ]:
md(f"## Can the model reproduce spatial patterns of {layer_long} {vv_longname}?")

md(f"The ability of the model to reproduce spatial patterns of {layer_long} {vv_name} was assessed by comparing the modelled and observed {vv_name} at each grid cell. We calculated the Pearson correlation coefficient between the modelled and observed {vv_name} at each grid cell.")
md("This was carried out monthly and using the annual mean in each grid cell")

In [ ]:
ff = glob.glob(f"data_dir_value/oceanval_matchups/gridded/{variable}/**_{variable}*surface.nc")
ff = [x for x in ff if f"{source}_" in x]
if len(ff) != 1:
    raise ValueError("Something is wrong with the file")
layer = os.path.basename(ff[0]).split("_")[-1].replace(".nc", "")
ds_cor = nc.open_data(ff)
ds_cor.subset(lon = lon_lim, lat = lat_lim)
ds_cor.set_precision("F32")
ds_cor.tmean("month")
ds_cor.cor_space("model", "observation")
ds_cor_df = ds_cor.to_dataframe().reset_index()
ds_cor_df = ds_cor_df.dropna()
time_name = [x for x in list(ds_cor.to_xarray().coords) if "time" in x][0]
# rename time in dataframe
ds_cor_df.rename(columns = {time_name: "time"}, inplace = True)
# extract the month
ds_cor_df["month"] = ds_cor_df.time.dt.month
ds_cor_df = ds_cor_df.loc[:,["month", "cor"]].drop_duplicates()
# change month number to month name
ds_cor_df["month"] = ds_cor_df["month"].apply(lambda x: calendar.month_abbr[x])
# now do this annually
ds_cor = nc.open_data(ff)
ds_cor.subset(lon = lon_lim, lat = lat_lim)
ds_cor.set_precision("F32")
ds_cor.tmean("month")
ds_cor.tmean()
df_annual = ds_cor.to_dataframe()
ds_cor.cor_space("model", "observation")
ds_cor_df_annual = ds_cor.to_dataframe().reset_index()
ds_cor_df_annual = ds_cor_df_annual.dropna()
time_name = [x for x in list(ds_cor.to_xarray().coords) if "time" in x][0]
# rename time in dataframe
ds_cor_df_annual.rename(columns = {time_name: "time"}, inplace = True)
# extract the month
ds_cor_df_annual["month"] = ds_cor_df_annual.time.dt.month
ds_cor_df_annual = ds_cor_df_annual.loc[:,["month", "cor"]].drop_duplicates()
# output to csv
ds_cor_df_annual = ds_cor_df_annual.assign(month = "Annual mean")
# merge the two dataframes
ds_cor_df = pd.concat([ds_cor_df_annual, ds_cor_df])
# change month to period
ds_cor_df.rename(columns = {"month": "period"}, inplace = True)
# Give the columns more sensible names
ds_cor_df.rename(columns = {"cor": "Correlation coefficient"}, inplace = True)
ds_cor_df.rename(columns = {"period": "Time period"}, inplace = True)
# drop any na
ds_cor_df.dropna(inplace = True)
df_display(ds_cor_df)

In [ ]:
md(f"**Table {i_table}**: Pearson correlation coefficient between modelled and observed {layer_long} {vv_longname} at each grid cell. The correlation was calculated monthly and using the annual mean in each grid cell.")
i_table += 1

In [ ]:
time_series = False
if regional:
    ff = glob.glob(f"data_dir_value/oceanval_matchups/gridded/{variable}/**_{variable}_*surface.nc")
    ff = [x for x in ff if f"{source}_" in x]
    ds_ts = nc.open_data(ff)
    ds_ts.subset(lon = lon_lim, lat = lat_lim)
    years = ds_ts.years
    year_range = f"{min(years)}-{max(years)}"
    if len(years) > 1:
        ds_ts.tmean("year")
        ds_ts.run()
        time_series = True

In [ ]:
if time_series:
    md(f"The ability of the model to reproduce multi-year trends in {layer_long} {vv_name} was assessed by comparing the modelled and observed time series of annual {vv_name} across each region.")
    md(f"The figure below shows the average {vv_name} in each region")

In [ ]:
if time_series:
    df_all = []
    for vv in ds_regions.variables:
        ds_rr = ds_regions.copy()
        ds_rr.subset(variable = vv)
        ds_rr.run()
        ds_vv = ds_ts.copy()
        ds_vv * ds_rr
        ds_region = ds_vv.copy()
        ds_vv.spatial_mean()
        region = list(regions_contents.query("variable == @vv").long_name)[0]
        time_name = [x for x in list(ds_vv.to_xarray().coords) if "time" in x][0]
        df_vv = (
            ds_vv
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .melt("time")
            .assign(year = lambda x: x.time.dt.year)
            .assign(region = vv)
        )
        df_all.append(df_vv)
        ds_region.tmean()
        df_region = (
            ds_region
            .to_dataframe()
            .dropna()
            .reset_index()
            .loc[:,["model", "observation"]]
            .drop_duplicates()
        )
    
        del ds_rr, ds_vv, ds_region
    df_all = pd.concat(df_all).dropna()
        
    df_all = (
        df_all
        .merge(df_mapped.loc[:,["long_name", "variable"]].drop_duplicates().rename(columns = {"variable": "region"}))
    )

In [ ]:
%%capture --no-display
%%R -i time_series -i variable -i df_all
if(time_series){
library(tidyverse)

ylab = str_glue("Spatial average {variable} ({units})")
ylab <- str_replace(ylab, "/m\\^3", "m<sup>-3</sup>")
ylab <- str_replace(ylab, "/m\\^2", "m<sup>-2</sup>")
# handl /yr
ylab <- str_replace(ylab, "/yr", " yr<sup>-1</sup>")
# handle /m2
ylab <- str_replace(ylab, "/m2", "m<sup>2</sup>")
# handle /m3
ylab <- str_replace(ylab, "/m3", "m<sup>3</sup>")
# fix /kg to kg^-1
ylab <- str_replace(ylab, "/kg", "kg<sup>-1</sup>")
# CO2
ylab <- str_replace(ylab, "CO2", "CO<sub>2</sub>")
# make variable title
# pco2
ylab <- str_replace(ylab, "pCO2", "pCO<sub>2</sub>")
ylab <- str_replace(ylab, "pco2", "pCO<sub>2</sub>")
# mutam should use greek letters
ylab <- str_replace(ylab, "muatm", "µatm")

# create a suitable scale_x_continuous based on year
# get the min and max year
min_year = min(df_all$year)
max_year = max(df_all$year)
# create a sequence of years, maximum is 7
ceiling((max_year - min_year)/7)
seq_year <- seq(min_year, max_year, ceiling((max_year - min_year)/7))


gg <- df_all %>%
    ggplot(aes(x = year, y = value, colour = variable))+
    geom_line()+
    facet_wrap(~long_name)+
    labs(y = ylab)+
    labs(x = "Year")+
    theme(legend.position = "top")+
    scale_color_manual(values = c("red", "blue"))+
    theme_bw(base_size = 10)+
    labs(colour = "")+
    theme(legend.position = "top")+
    theme(axis.title.y = ggtext::element_markdown())+
    scale_color_fivethirtyeight()+
    scale_x_continuous(breaks = seq_year, labels = seq_year)

    gg


}

In [ ]:
# Now do the verticals if needed
if os.path.exists(ff_vertical):
    verticals = True
    md(f"## How well does the model reproduce vertical profiles of {vv_longname}?")
    text = f"The ability of the model to reproduce vertical profiles of {vv_name} was assessed by comparing the modelled and observed vertical profiles of {vv_name}. This was carried out by calculating the zonal average vertical profile of {vv_name} for both the model and observation based on annual means."
    md(text)
else:
    verticals = False

In [ ]:
try:
    regions = ds_regions.variables
    # coerce to list
    regions = list(regions)
    regions.append("full_domain")
    regionals = True
except:
    regionals = True
    regions = ["full_domain"]

In [ ]:
if time_series:
    md(f"**Figure {i_figure}**: Changes in {layer_long} {vv_longname} for model and observations for each region for the period {year_range}. The spatial average is taken over the region.") 
    i_figure += 1

In [ ]:
if verticals:
    ds_verticals = nc.open_data(ff_vertical, checks = False)
    ds_verticals.tmean()
    ds_verticals.run()
    levels = ds_verticals.levels
    max_depth = max(levels)
    min_depth = min(levels)
    # 100 appropriate levels, evenly spaces
    new_levels = np.linspace(min_depth, max_depth, 100)
    ds_verticals.vertical_interp(new_levels, fixed = True)

In [ ]:
if verticals:
    df_zonals = []
    for rr in regions:
        ds_rr = ds_verticals.copy()
        if rr != "full_domain":
            ds_subset = ds_regions.copy()
            ds_subset.as_missing(0)
            ds_subset.set_fill(-9999)
            ds_subset.subset(variable = rr)
            ds_subset.run()
            long_name = ds_subset.contents.long_name[0]
            ds_subset - 1
            ds_rr + ds_subset
            ds_rr.zonal_mean()
        else:
            ds_rr.zonal_mean()
            long_name = "Full domain"
        df_merged = ds_rr.to_dataframe().reset_index(drop = True)
        depth_name = [x for x in ds_rr.to_xarray().coords if "depth" in x][0]
        # rename
        df_merged = df_merged.rename(columns = {depth_name: "depth"})
        lon_name = [x for x in ds_rr.to_xarray().coords if "lon" in x][0]
        lat_name = [x for x in ds_rr.to_xarray().coords if "lat" in x][0]
        # ditch lon
        df_merged = df_merged.drop(columns = [lon_name])
        # rename lat to lat
        df_merged = df_merged.rename(columns = {lat_name: "lat"})
        df_merged = (
            df_merged
            # melt it
            .melt(id_vars = ["depth", "lat"], value_vars = ["model", "observation"])
        )
        df_merged = (df_merged
        # variable as title
            .assign(variable = lambda x: x.variable.str.title())
        ).assign(region = long_name) 
        df_zonals.append(df_merged)
    df_zonals = pd.concat(df_zonals).dropna().reset_index(drop = True)

    # Put Full domain first
    df_zonals = pd.concat(
        [df_zonals.query("region == 'Full domain'"),
        df_zonals.query("region != 'Full domain'")]

    )

    
    # only lat, model and observation
    # df_rr = df_rr.loc[:,[lat_name, "depth", "model", "observation"]].dropna()
else:
    df_zonals = pd.DataFrame({"foo":["bar"]})

In [ ]:
%%R -i verticals -i regionals -i df_zonals -i vv_name -i units -w 800 -h zonal_height
# is df_zonals a dataframe

if("region" %in% colnames(df_zonals)){
    library(tidyverse)
    units <- str_replace(units, "/m\\^3", "m<sup>-3</sup>")
    units <- str_replace(units, "/m\\^2", "m<sup>-2</sup>")
    units <- str_replace(units, "/yr", " yr<sup>-1</sup>")
    units <- str_replace(units, "/m2", "m<sup>-2</sup>")
    units <- str_replace(units, "/m3", "m<sup>-3</sup>")
    units <- str_replace(units, "/kg", "kg<sup>-1</sup>")
    # CO2

    # loop through the regions
    i <- 1
    big_list = list()
    for(rr in unique(df_zonals$region)){

    gg <- df_zonals %>%
        filter(region == rr) %>%
        drop_na() %>%
        ggplot()+
        geom_raster(aes(x = lat, y = depth, fill = value))+
        facet_wrap(~variable)+
        scale_y_reverse()+
        scale_fill_viridis_c(guide  = guide_colourbar(title.position = "right"))+
        theme_bw(base_size = 10)+
        theme(legend.title = ggtext::element_markdown(angle = -90), legend.title.align = 0.5)+
        labs(x = "Latitude", y = "Depth (m)", fill = str_glue("{vv_name} ({units})"))+
        theme(axis.title.y = ggtext::element_markdown())+
        theme(axis.title.x = ggtext::element_markdown())+
        coord_cartesian(expand = FALSE)+
        # ditch the x axis labels

        labs(x = "", title = rr)

    x_breaks <- ggplot_build(gg)$layout$panel_params[[1]]$x$breaks
    x_breaks <- na.omit(x_breaks)
    x_labels <- paste0(x_breaks, "°")
    # add N to x_label is x_break > 0
    x_labels <- ifelse(x_breaks > 0, paste0(x_breaks, "°N"), ifelse(x_breaks < 0, paste0(abs(x_breaks), "°S"), "0°"))
    # add S to x_label is x_break < 0
    x_labels <- ifelse(x_breaks < 0, paste0(abs(x_breaks), "°S"), x_labels)
    # remove  the str "-" from x_labels if x_break < 0
    x_labels <- gsub("-", "", x_labels)

# no missing values
    gg1 <- gg + scale_x_continuous(breaks = x_breaks, labels= x_labels)

    # now do a bias plot
    df_bias <- df_zonals %>%
        filter(region == rr) %>%
        drop_na() %>%
        pivot_wider(names_from = variable, values_from = value) %>%
        mutate(bias = Model - Observation)
    gg2 <- df_bias %>%

        ggplot()+
        geom_raster(aes(x = lat, y = depth, fill = bias))+
        scale_y_reverse()+
        scale_fill_gradient2(low = "blue", mid = "white", high = "red", 
                             guide  = guide_colourbar(title.position = "right"))+
        theme_bw(base_size = 10)+
        theme(legend.title = ggtext::element_markdown(angle = -90), legend.title.align = 0.5)+
        labs(x = "Latitude", y = "Depth (m)", fill = str_glue("Bias ({model_unit})"))+
        theme(axis.title.y = ggtext::element_markdown())+
        theme(axis.title.x = ggtext::element_markdown())+
        coord_cartesian(expand = FALSE)
        # ditch the x axis labels
    gg2 <- gg2 + labs(x = "")
    x_breaks <- ggplot_build(gg2)$layout$panel_params[[1]]$x$breaks
    x_breaks <- na.omit(x_breaks)
    x_labels <- paste0(x_breaks, "°")
    # add N to x_label is x_break > 0
    x_labels <- ifelse(x_breaks > 0, paste0(x_breaks, "°N"), ifelse(x_breaks < 0, paste0(abs(x_breaks), "°S"), "0°"))
    # add S to x_label is x_break < 0
    x_labels <- ifelse(x_breaks < 0, paste0(abs(x_breaks), "°S"), x_labels)
    # remove  the str "-" from x_labels if x_break < 0
    x_labels <- gsub("-", "", x_labels)


    gg2 <- gg2 + scale_x_continuous(breaks = x_breaks, labels= x_labels)

# combine the two plots, use rr as title
    gg <- cowplot::plot_grid(gg1, gg2, ncol = 1)




    big_list[[i]] <- gg
    i <- i + 1
    
    }

    cowplot::plot_grid(plotlist = big_list, ncol = 1)


}

In [ ]:
if verticals and regionals:
    md(f"**Figure {i_figure}**: Vertical profiles of {layer_long} {vv_longname} for model and observations for each region. The zonal average is taken over the region.")
    i_figure += 1

In [ ]:
chunk_end

In [ ]:
md(f"## Data Sources for validation of {vv_name}")
md_basic(definitions[variable].sources[source])

In [ ]:
if test_status:
    md("This is getting to the end!")